In [1]:
import bs4
import pandas
import requests
import re

In [2]:
def get_car_info(url):
    page = requests.get(url, headers={"Accept-Language": "en-US"})
    soup = bs4.BeautifulSoup(page.text, "html.parser")
    car_info = {}
    car_info1={}
    gia = soup.find(class_='styles__PriceWrapper-sc-14jh840-3 kCmIqn').text.split(" ")[0]
    car_info1['Gia(VND)'] = gia


    idinfo={0:'Hãng', 1:'Dòng xe', 2:'Năm sản xuất', 3:'Số Km đã đi', 4:'Tình trạng', 5:'Hộp số', 6:'Nhiên liệu',
            7:'Xuất xứ', 8:'Kiểu dáng', 9:'Số chỗ', 10:'Người bán gửi Video', 11:'Dia_chi'}
    for feature in idinfo:
      car_info[idinfo[feature]]= 'null'
    for title in soup.findAll('div', class_='media-body media-middle'):
      if (len(title.text.split(': ')) > 1 ):
        car_info[title.text.split(': ')[0]] = title.text.split(': ')[1]
      else:
        car_info['Dia_chi'] = title.text

    for k in range(0, 12):
      car_info1[idinfo[k]]= car_info[idinfo[k]]
    return car_info1

In [ ]:
def get_link(url, data):
      page = requests.get(url,headers={"Accept-Language":"en-US"})
      soup= bs4.BeautifulSoup(page.text,"html.parser")
      header = ['Gia(VND)', 'Hãng', 'Dòng xe', 'Năm sản xuất', 'Số Km đã đi', 'Tình trạng', 'Hộp số', 'Nhiên liệu',
            'Xuất xứ', 'Kiểu dáng', 'Số chỗ', 'Người bán gửi Video', 'Dia_chi']
      for link in soup.find_all(class_="styles__AdItemLayout-sc-1s892rt-0 gQrapR"):
          car_url = re.findall(r'''href=\n"\S+\"''', str(link))[0]
          car_url= re.sub(r'href=\"', '', str(car_url))
          car_url= re.sub(r'\"','',car_url)
          car_info= get_car_info(main_url + car_url)
          car_data= [car_info[fea] for fea in header]
          data.append(car_data)

In [ ]:
data=[]
for i in range(1, 10):
  print(i)
  get_link('https://oto.com/mua-ban-oto?page=' + str(i), data)

In [ ]:
df= pandas.DataFrame(data)
df.to_csv('./raw_data/crawl.csv', index= False)